In [1]:
from axi_stream_driver import NeuralNetworkOverlay
import numpy as np
import time

X_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

print(X_test.shape)
print(X_test[0].shape)

print(y_test.shape)



# print(X_test[0])

nn = NeuralNetworkOverlay('design_1.bit', X_test[0:1].shape, y_test[0:1].shape)

start = time.time()
y_hw, latency, throughput = nn.predict(X_test[0:1], profile=True)
end = time.time()

print(end-start)
print(latency)

(9, 80, 80, 3)
(80, 80, 3)
(9, 2)


Classified 1 samples in 0.0031379999999999997 seconds (318.6743148502231 inferences / s)
0.004920244216918945
0.0031379999999999997


In [33]:
from axi_stream_driver import NeuralNetworkOverlay
import numpy as np
import time

X_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

print(X_test.shape)
print(X_test[0].shape)

print(y_test.shape)



# print(X_test[0])

nn = NeuralNetworkOverlay('design_1.bit', X_test[0:1].shape, y_test[0:1].shape)

start = time.time()
y_hw, latency, throughput = nn.predict(X_test[0:1], profile=False)
end = time.time()

print(end-start)
print(latency)

(9, 80, 80, 3)
(80, 80, 3)
(9, 2)


ValueError: not enough values to unpack (expected 3, got 1)

In [35]:
from axi_stream_driver import NeuralNetworkOverlay
import numpy as np

# 1) Load your test data
X_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

# 2) Instantiate overlay (shapes passed positionally)
nn = NeuralNetworkOverlay('design_1.bit',
                          X_test[0:1].shape,
                          y_test[0:1].shape)

# 3) Grab the AXI‑Timer instance from the overlay
timer = nn.overlay.ip_dict['axi_timer_0']

# 4) Reset and start the timer
timer.reset()
timer.start()

# 5) Run one inference (no internal profiling)
y_hw = nn.predict(X_test[0:1], profile=False)

# 6) Stop the timer and read out raw cycle count
timer.stop()
cycles = timer.read()
print(f"Raw PL cycles for one inference: {cycles}")

# 7) Convert cycles → time (assumes 100 MHz PL clock)
pl_clk_hz = 100e6
time_s = cycles / pl_clk_hz
print(f"Measured PL execution time: {time_s*1e3:.3f} ms")


AttributeError: Could not find IP or hierarchy overlay in overlay

In [4]:
def predict(self, input_data, profile=False):
    print("started")
    import time

    # Assumes input_data is a NumPy array with shape (1, ...)
    assert input_data.shape[0] == 1, "This version handles batch size 1 for simplicity"

    # Copy data to input buffer
    np.copyto(self.input_buffer, input_data)

    # Start input DMA transfer
    self.dma.sendchannel.transfer(self.input_buffer)
    self.dma.sendchannel.wait()

    # Start the hardware accelerator (PL compute)
    self.ip.write(0x00, 0x01)  # ap_start = 1

    # Start timing for PL compute
    pl_start = time.time()

    # Wait for ap_done = 1 (bit 2 of control register)
    while (self.ip.read(0x00) & 0x4) == 0:
        pass

    pl_end = time.time()
    pl_latency = pl_end - pl_start  # in seconds

    # Start output DMA transfer
    self.dma.recvchannel.transfer(self.output_buffer)
    self.dma.recvchannel.wait()

    # Convert output to NumPy
    output_array = self.output_buffer.copy()

    if profile:
        throughput = 1.0 / pl_latency  # samples per second
        return output_array, pl_latency, throughput
    else:
        return output_array


In [23]:
y_hw, latency, throughput = nn.predict(X_test[0:1], profile=True)
print(f"PL compute latency: {latency * 1000:.3f} ms")
print(f"Throughput: {throughput:.2f} samples/sec")


Classified 1 samples in 0.00649 seconds (154.08320493066256 inferences / s)
PL compute latency: 6.490 ms
Throughput: 154.08 samples/sec


In [24]:
from pynq import Overlay, allocate
import numpy as np
import time

class NeuralNetworkOverlay:
    def __init__(self, bitfile, input_shape, output_shape):
        # Load the overlay
        self.overlay = Overlay(bitfile)
        self.ip = self.overlay.my_nn_ip_name  # CHANGE THIS to your actual IP name
        self.dma = self.overlay.axi_dma_0     # CHANGE this if your DMA is named differently

        # Create input/output buffers in contiguous memory
        self.input_buffer = allocate(shape=input_shape, dtype=np.float32)
        self.output_buffer = allocate(shape=output_shape, dtype=np.float32)

    def predict(self, input_data, profile=False):
        assert input_data.shape[0] == 1, "This version supports batch size = 1 only"

        # Copy data into input buffer
        np.copyto(self.input_buffer, input_data)

        # Start DMA input transfer
        self.dma.sendchannel.transfer(self.input_buffer)
        self.dma.sendchannel.wait()

        # Start the hardware accelerator
        self.ip.write(0x00, 0x01)  # ap_start = 1

        # Measure PL computation time
        pl_start = time.time()
        while (self.ip.read(0x00) & 0x4) == 0:  # Wait for ap_done (bit 2)
            pass
        pl_end = time.time()
        pl_latency = pl_end - pl_start

        # Start DMA output transfer
        self.dma.recvchannel.transfer(self.output_buffer)
        self.dma.recvchannel.wait()

        # Copy result
        output_array = self.output_buffer.copy()

        if profile:
            throughput = 1.0 / pl_latency  # samples/sec
            return output_array, pl_latency, throughput
        else:
            return output_array


In [25]:
X_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

print(X_test.shape)
print(X_test[0].shape)

nn = NeuralNetworkOverlay('design_1.bit', X_test[0:1].shape, y_test[0:1].shape)

y_hw, latency, throughput = nn.predict(X_test[0:1], profile=True)

print(f"PL compute latency: {latency * 1000:.3f} ms")
print(f"Throughput: {throughput:.2f} samples/sec")
print("Output:", y_hw)


(9, 80, 80, 3)
(80, 80, 3)


AttributeError: Could not find IP or hierarchy my_nn_ip_name in overlay

In [26]:
from pynq import Overlay, allocate
import numpy as np
import time

class NeuralNetworkOverlay:
    def __init__(self, bitfile, input_shape, output_shape):
        # Load the overlay
        self.overlay = Overlay(bitfile)
        
        # Get IP and DMA from overlay
        self.ip = self.overlay.myproject_axi_0     # Your HLS IP
        self.dma = self.overlay.axi_dma_0          # Your DMA engine

        # Allocate contiguous memory for input/output buffers
        self.input_buffer = allocate(shape=input_shape, dtype=np.float32)
        self.output_buffer = allocate(shape=output_shape, dtype=np.float32)

    def predict(self, input_data, profile=False):
        assert input_data.shape[0] == 1, "Only batch size 1 is supported in this version"

        # Copy input data to DMA buffer
        np.copyto(self.input_buffer, input_data)

        # Start input DMA transfer
        self.dma.sendchannel.transfer(self.input_buffer)
        self.dma.sendchannel.wait()

        # Start accelerator by asserting ap_start
        self.ip.write(0x00, 0x01)  # ap_start = 1

        # Measure PL compute time
        pl_start = time.time()
        while (self.ip.read(0x00) & 0x4) == 0:  # Wait for ap_done (bit 2)
            pass
        pl_end = time.time()
        pl_latency = pl_end - pl_start

        # Start output DMA transfer
        self.dma.recvchannel.transfer(self.output_buffer)
        self.dma.recvchannel.wait()

        # Get output as NumPy array
        output_array = self.output_buffer.copy()

        if profile:
            throughput = 1.0 / pl_latency
            return output_array, pl_latency, throughput
        else:
            return output_array


In [27]:
import numpy as np

X_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

nn = NeuralNetworkOverlay('design_1.bit', X_test[0:1].shape, y_test[0:1].shape)

# Inference on one sample, with timing
y_hw, latency, throughput = nn.predict(X_test[0:1], profile=True)

print(f"PL compute latency: {latency * 1000:.3f} ms")
print(f"Throughput: {throughput:.2f} samples/sec")
print("Output:", y_hw)


AttributeError: Could not find IP or hierarchy myproject_axi_0 in overlay

In [28]:
from pynq import Overlay, allocate
import numpy as np
import time

class NeuralNetworkOverlay:
    def __init__(self, bitfile, input_shape, output_shape):
        self.overlay = Overlay(bitfile)

        # Access IP and DMA inside hierarchy
        self.ip = self.overlay.hier_0.myproject_axi_0
        self.dma = self.overlay.hier_0.axi_dma_0

        # Allocate DMA buffers
        self.input_buffer = allocate(shape=input_shape, dtype=np.float32)
        self.output_buffer = allocate(shape=output_shape, dtype=np.float32)

    def predict(self, input_data, profile=False):
        assert input_data.shape[0] == 1, "Only batch size 1 is supported"

        # Copy data to input buffer
        np.copyto(self.input_buffer, input_data)

        # Start DMA input transfer
        self.dma.sendchannel.transfer(self.input_buffer)
        self.dma.sendchannel.wait()

        # Start the accelerator
        self.ip.write(0x00, 0x01)  # ap_start = 1

        # Measure PL compute time
        pl_start = time.time()
        while (self.ip.read(0x00) & 0x4) == 0:  # Wait for ap_done
            pass
        pl_end = time.time()
        pl_latency = pl_end - pl_start

        # Start DMA output transfer
        self.dma.recvchannel.transfer(self.output_buffer)
        self.dma.recvchannel.wait()

        output_array = self.output_buffer.copy()

        if profile:
            throughput = 1.0 / pl_latency
            return output_array, pl_latency, throughput
        else:
            return output_array


In [29]:
import numpy as np

X_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

nn = NeuralNetworkOverlay('design_1.bit', X_test[0:1].shape, y_test[0:1].shape)

# Inference on one sample, with timing
y_hw, latency, throughput = nn.predict(X_test[0:1], profile=True)

print(f"PL compute latency: {latency * 1000:.3f} ms")
print(f"Throughput: {throughput:.2f} samples/sec")
print("Output:", y_hw)


AttributeError: Could not find IP or hierarchy myproject_axi_0 in overlay

In [36]:
from pynq import Overlay
from pynq.lib import AxiGPIO

overlay = Overlay("design_1.bit")

# Access the timer
timer = overlay.axi_timer_0  # or whatever name you gave it

# Start the timer before prediction
timer.write(0x00, 0x110)  # Load + Enable down count + Enable timer
y_hw = overlay.myproject_axi_0.predict(X_test[0:1])  # Replace with your actual call
timer.write(0x00, 0x0)  # Stop the timer

# Read the timer count
elapsed = timer.read(0x08)  # Timer counter register
clk_freq = 100e6  # 100 MHz for example
pl_time_sec = elapsed / clk_freq
print(f"PL Execution Time: {pl_time_sec * 1e6:.2f} µs")


AttributeError: Could not find IP or hierarchy axi_timer_0 in overlay